In [ ]:
# stdlib
import logging
from pathlib import Path
from pprint import pprint
import random
import sys
import time

# third party
import veilid

project_root = Path.cwd().parent.parent.parent
veilid_path = project_root / "packages" / "grid" / "veilid" / "server"
sys.path.append(veilid_path.as_posix())

# third party
from veilid_streamer import VeilidStreamer  # type: ignore

logging.getLogger('veilid_streamer').setLevel(logging.DEBUG)

In [ ]:
host = "localhost"
port = 5960

In [ ]:
async def noop_callback(update: veilid.VeilidUpdate):
    pass


conn = await veilid.json_api_connect(host, port, noop_callback)

In [ ]:
router = await (await conn.new_routing_context()).with_default_safety()
dht_key_str = input("Enter DHT Key of the receiver: ")
dht_key = veilid.TypedKey(dht_key_str.lstrip("VLD0:"))
await router.open_dht_record(key=dht_key, writer=None)
record_value = await router.get_dht_value(key=dht_key, subkey=0, force_refresh=True)
private_route = await conn.import_remote_private_route(record_value.data)

In [ ]:
vs = VeilidStreamer(connection=conn, router=router)
benchmarks = {}

await router.app_message(private_route, b"Starting stream...")
for message_size_kb in range(0, 13):  # Powers of two from 1 to 4096
    message_size_kb = 2**message_size_kb
    message = random.randbytes(message_size_kb * 1024)
    print(f"Sending message of size {len(message)} bytes")
    start = time.time()
    response = await vs.stream(private_route, message)
    end = time.time()
    print(f"[{end - start}s] Response: {response}")
    benchmarks[message_size_kb] = end - start
await router.app_message(private_route, b"QUIT")

pprint(benchmarks)